In [16]:
import boto3
import os,sys
import datetime as dt
import glob
import pandas as pd
pd.set_option('display.max_rows', 1000)
import hvplot.pandas
import holoviews as hv
import subprocess as sp

In [49]:
os.makedirs('/dev/shm/logs',exist_ok=True)
os.chdir('/dev/shm/logs')

In [50]:
!aws s3 sync s3://earthbigdata/logs/ /dev/shm/logs --quiet

In [51]:
def make_entry(ip_dict,date_dict,logfile):  
    with open(logfile,"r") as f:
        lines=f.readlines()
    date = lines[0].split('[')[1].split('+')[0].strip()
    date = dt.datetime.strptime(date,'%d/%b/%Y:%H:%M:%S')
    ipaddr=lines[0].split(']')[1].split('-')[0].strip()
    if not ipaddr in ip_dict:
        ip_dict[ipaddr]=[]
    if not date.date() in date_dict:
        date_dict[date.date()]=0
    ip_dict[ipaddr].append(date)
    date_dict[date.date()]+=1
    

In [52]:
ip_dict={}
date_dict={}

for log in glob.glob('ebd-covid19*'):
    make_entry(ip_dict,date_dict,log)

In [53]:
access=pd.DataFrame.from_dict(date_dict,orient='index')

In [54]:
access.hvplot.barh(padding=0.1,grid=True,ylabel='Access Times per day',xlabel='',title='EBD COVID 19 Access')

:Bars   [index]   (0)

In [55]:
ip_count={}
for i in ip_dict:
    ip_count[i]=len(ip_dict[i])

In [56]:
access_count=pd.DataFrame.from_dict(ip_count,orient='index')

access_count=access_count[access_count>=5].dropna()

access_count = access_count.sort_values(0,ascending=False)

access_count['nslookup']=''
for i in access_count.index:
    cmd='nslookup {}'.format(i)
    try:
        ret= sp.check_output(cmd.split())
        nslookup = ret.decode().split('name = ')[1].split('\n')[0].strip('.')
        # print(nslookup)
        access_count.loc[i,'nslookup']=nslookup
    except:
        pass

In [57]:
access_count.head(20)

,0,nslookup
24.218.227.219,378.0,c-24-218-227-219.hsd1.ma.comcast.net
52.44.93.197,95.0,ec2-52-44-93-197.compute-1.amazonaws.com
34.231.157.157,91.0,ec2-34-231-157-157.compute-1.amazonaws.com
73.23.120.180,84.0,c-73-23-120-180.hsd1.fl.comcast.net
34.232.127.140,73.0,ec2-34-232-127-140.compute-1.amazonaws.com
71.232.190.220,60.0,c-71-232-190-220.hsd1.ma.comcast.net
72.205.80.22,58.0,ip72-205-80-22.sb.sd.cox.net
93.218.246.5,55.0,p5DDAF605.dip0.t-ipconnect.de
70.191.91.13,51.0,ip70-191-91-13.sb.sd.cox.net
73.143.234.232,50.0,c-73-143-234-232.hsd1.ma.comcast.net


In [58]:
len(list(ip_dict.keys()))

1898

In [59]:
list(ip_dict.keys())[:10]

['24.94.16.143',
 '142.234.50.218',
 '66.215.238.104',
 '76.110.137.124',
 '68.46.172.194',
 '52.44.93.197',
 '24.218.227.219',
 '108.218.159.111',
 '72.205.80.22',
 '76.126.182.37']

In [61]:
for i in list(ip_dict.keys()):
    if ip_dict[i][-1].date() == dt.datetime.now().date():
        if i in list(access_count.index):
            print(ip_dict[i][-1].date(),access_count.loc[i][0],'\t',access_count.loc[i].nslookup)

2020-03-30 6.0 	 255.sub-174-195-193.myvzw.com
2020-03-30 5.0 	 150.178.94.184.cpe.quadro.net
2020-03-30 5.0 	 cpe-104-174-246-74.socal.res.rr.com
2020-03-30 9.0 	 CPE00fc8d3bf703-CM00fc8d3bf700.cpe.net.cable.rogers.com
2020-03-30 8.0 	 abts-north-dynamic-090.134.162.122.airtelbroadband.in
2020-03-30 5.0 	 76-216-241-202.lightspeed.irvnca.sbcglobal.net
2020-03-30 16.0 	 75.sub-174-195-205.myvzw.com
